In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import scipy.optimize as scopt
import glob
import os

%matplotlib inline

## Setup

In [2]:
def load_json(file):
    '''
    Loads txt file and returns dataframe using the json dict
    '''
    data = {}
    with open(file, encoding='utf-8') as f:
        for line in f:
            d = str(line).replace("'", '"')
            data = json.loads(d)

    return pd.DataFrame(data)

In [3]:
os.chdir(os.getcwd()+'/rss-data')

## Retrieving and loading each trace

In [4]:
files = [file for file in glob.glob("*.txt")]
dfs = [load_json(file) for file in files]

## Preprocessing data & Separating traces by transmitter/MAC

In [5]:
def preprocess(df_list):
    '''
    Computes rolling average for RSS for each trace
    '''
    clean_dfs = []
    for df in df_list:
        df['rss'] = pd.to_numeric(df['rss'])
        df['mean_rss'] = df['rss'].rolling(window=5, win_type='triang').mean()
        df = df.dropna()
        clean_dfs.append(df)
    macs = list(set(df['mac']))
    
    return clean_dfs, macs

## Defining functions for scipy.optimize.curve_fit

In [6]:
def localization(coords, c, g, x0, y0):
    x1, y1 = coords
    return c + g * np.log10(((x1-x0)**2 + (y1-y0)**2)**0.5)

In [7]:
def c_fit(tx):
    popt, pcov = scopt.curve_fit(localization,
                                 (tx.loc_x,tx.loc_y),
                                 tx.rss,
                                 maxfev=250000,
                                 bounds=((-60,2,-20,-20),(30,6,20,20)))
    return popt

In [8]:
def fit(df_list, trace_num=0):
    '''
    Iterates through traces to estimate missing parameters
    '''
    clean_dfs, macs = preprocess(df_list)
    df = clean_dfs[trace_num]
    popts = []
    for mac in macs:
        tx = df[df['mac'] == mac]
        popts.append([str(mac)] + list(c_fit(tx)))
        
    return pd.DataFrame(popts, columns=['mac','c', 'gamma', 'x', 'y'])

## Fitting the data
We iterate through each data trace and get estimates for [C, $\gamma$, x, y]. We then use the known locations of two transmitters to test our model's accuracy. 

```x1, y1 = (6.8, 6.8)
x2, y2 = (-.87, 9.45)```


In [9]:
def go(df_list):
    results = []
    for i in range(len(df_list)):
        results.append(fit(df_list, i))
        
    res_df = pd.concat(results)
    return res_df

In [10]:
fit(dfs, 0)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.083633,2.072193,6.420000,3.647195
1,8c:85:90:16:0a:a4,-58.290314,6.000000,0.262613,6.252222
2,ac:9e:17:7d:31:e8,-52.615062,6.000000,6.377578,3.798023
3,f8:cf:c5:97:e0:9e,-60.000000,2.000000,6.419792,5.503429


In [11]:
fit(dfs, 1)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.957415,2.004767,6.420000,2.881014
1,8c:85:90:16:0a:a4,-57.084796,6.000000,0.279833,6.911146
2,ac:9e:17:7d:31:e8,-53.306605,6.000000,6.445779,3.381894
3,f8:cf:c5:97:e0:9e,-59.994626,2.000437,2.604670,6.802287


In [12]:
fit(dfs, 2)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.742243,2.071617,6.420000,4.673980
1,8c:85:90:16:0a:a4,-57.594846,6.000000,1.535323,6.772372
2,ac:9e:17:7d:31:e8,-54.465843,6.000000,6.539346,4.221144
3,f8:cf:c5:97:e0:9e,-60.000000,2.000000,4.567288,6.785382


In [13]:
fit(dfs, 3)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.864824,2.009999,6.419999,3.698717
1,8c:85:90:16:0a:a4,-55.425762,3.288460,-0.318745,6.823934
2,ac:9e:17:7d:31:e8,-52.429625,6.000000,6.430571,4.007762
3,f8:cf:c5:97:e0:9e,-59.998994,2.000207,6.419835,5.992509


In [14]:
fit(dfs, 4)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-60.000000,2.000000,4.189498,6.784485
1,8c:85:90:16:0a:a4,-58.571653,5.666728,-0.123028,6.819489
2,ac:9e:17:7d:31:e8,-52.875939,6.000000,6.453224,3.602943
3,f8:cf:c5:97:e0:9e,-59.997168,2.000202,6.420000,3.449284


In [15]:
fit(dfs, 5)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.999994,2.003661,5.825070,6.773060
1,8c:85:90:16:0a:a4,-56.999310,6.000000,-0.635528,7.024005
2,ac:9e:17:7d:31:e8,-52.402237,6.000000,6.430942,4.230503
3,f8:cf:c5:97:e0:9e,-60.000000,2.000000,2.848970,6.800106


In [16]:
fit(dfs, 6)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.999805,2.514367,6.420240,3.921428
1,8c:85:90:16:0a:a4,-55.808159,4.498357,-0.560009,6.818772
2,ac:9e:17:7d:31:e8,-54.140527,6.000000,6.512422,3.376388
3,f8:cf:c5:97:e0:9e,-59.996185,2.000330,6.419935,5.672038


In [17]:
fit(dfs, 7)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-57.984876,6.000000,6.305743,6.495404
1,8c:85:90:16:0a:a4,-55.414463,3.008479,-0.272381,6.826635
2,ac:9e:17:7d:31:e8,-54.885176,6.000000,6.468883,3.451301
3,f8:cf:c5:97:e0:9e,-60.000000,2.000000,3.683464,6.790861


In [18]:
fit(dfs, 8)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-60.000000,6.000000,6.362825,5.382261
1,8c:85:90:16:0a:a4,-55.462510,2.797827,0.503607,6.820055
2,ac:9e:17:7d:31:e8,-53.616301,6.000000,6.377833,3.680786
3,f8:cf:c5:97:e0:9e,-60.000000,2.000000,2.663603,6.801684


In [19]:
fit(dfs, 9)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-58.566244,6.000000,6.049033,6.630630
1,8c:85:90:16:0a:a4,-54.725024,3.444676,-0.168366,6.830215
2,ac:9e:17:7d:31:e8,-53.310678,6.000000,6.486169,3.692227
3,f8:cf:c5:97:e0:9e,-59.998529,2.000126,3.793968,6.792218


In [20]:
fit(dfs, 10)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-58.776993,2.311456,6.420000,4.295704
1,8c:85:90:16:0a:a4,-57.675687,6.000000,-0.336317,7.420254
2,ac:9e:17:7d:31:e8,-53.554598,6.000000,6.411830,4.160496
3,f8:cf:c5:97:e0:9e,-59.973926,2.001981,2.686420,6.801596


In [21]:
fit(dfs, 11)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-60.000000,5.484009,5.699385,6.697405
1,8c:85:90:16:0a:a4,-54.553956,2.105577,2.680713,6.801637
2,ac:9e:17:7d:31:e8,-52.631350,6.000000,6.434128,3.855701
3,f8:cf:c5:97:e0:9e,-59.992521,2.000351,2.861732,6.800112


In [22]:
fit(dfs, 12)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.775239,2.009028,6.029825,6.773302
1,8c:85:90:16:0a:a4,-58.025207,6.000000,0.217810,7.561367
2,ac:9e:17:7d:31:e8,-54.739062,6.000000,6.518885,3.902527
3,f8:cf:c5:97:e0:9e,-59.999857,2.000146,5.888945,6.773726


In [23]:
fit(dfs, 13)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.687810,2.020391,6.419999,4.111963
1,8c:85:90:16:0a:a4,-53.672767,2.929385,2.114636,6.834599
2,ac:9e:17:7d:31:e8,-51.632776,6.000000,6.464295,4.083890
3,f8:cf:c5:97:e0:9e,-60.000000,2.000000,5.521436,6.777968


In [24]:
fit(dfs, 14)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.993725,2.222866,6.420027,4.352866
1,8c:85:90:16:0a:a4,-56.592995,4.118271,-0.394613,6.826849
2,ac:9e:17:7d:31:e8,-50.711077,6.000000,6.441874,4.370960
3,f8:cf:c5:97:e0:9e,-59.999151,2.000642,6.419999,5.431198


In [25]:
fit(dfs, 15)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.999984,5.270658,6.419484,4.614716
1,8c:85:90:16:0a:a4,-57.460194,6.000000,-0.061697,7.555504
2,ac:9e:17:7d:31:e8,-51.977283,6.000000,6.478895,4.619793
3,f8:cf:c5:97:e0:9e,-60.000000,2.000000,4.668303,6.784755


In [26]:
fit(dfs, 16)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.897295,6.000000,6.451548,5.330205
1,8c:85:90:16:0a:a4,-55.611144,3.827739,-0.058711,6.824892
2,ac:9e:17:7d:31:e8,-52.680201,6.000000,6.385853,3.583743
3,f8:cf:c5:97:e0:9e,-60.000000,2.000000,3.183033,6.797363


In [27]:
fit(dfs, 17)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-59.352868,2.152470,6.419996,5.747714
1,8c:85:90:16:0a:a4,-55.395125,3.937757,1.266382,6.808756
2,ac:9e:17:7d:31:e8,-51.504703,6.000000,6.435831,4.405454
3,f8:cf:c5:97:e0:9e,-59.995500,2.000454,2.708467,6.801409


In [28]:
fit(dfs, 18)

,mac,c,gamma,x,y
0,d8:c4:6a:50:e3:b1,-55.906127,2.176566,6.42000,6.078111
1,8c:85:90:16:0a:a4,-54.998835,3.356545,1.56192,6.807862
2,ac:9e:17:7d:31:e8,-51.299679,6.000000,6.36936,3.771573
3,f8:cf:c5:97:e0:9e,-59.995726,2.000252,6.42000,6.731507


In [29]:
results_df = go(dfs)

### Finding average estimated parameters using all the traces

In [30]:
results_df.groupby(['mac'])['c'].mean()

mac
8c:85:90:16:0a:a4   -56.282250
ac:9e:17:7d:31:e8   -52.883091
d8:c4:6a:50:e3:b1   -59.399425
f8:cf:c5:97:e0:9e   -59.996957
Name: c, dtype: float64

In [31]:
results_df.groupby(['mac'])['gamma'].mean()

mac
8c:85:90:16:0a:a4    4.472621
ac:9e:17:7d:31:e8    6.000000
d8:c4:6a:50:e3:b1    3.280213
f8:cf:c5:97:e0:9e    2.000270
Name: gamma, dtype: float64

In [32]:
results_df.groupby(['mac'])['x'].mean()

mac
8c:85:90:16:0a:a4    0.394392
ac:9e:17:7d:31:e8    6.445458
d8:c4:6a:50:e3:b1    6.185930
f8:cf:c5:97:e0:9e    4.536835
Name: x, dtype: float64

In [33]:
results_df.groupby(['mac'])['y'].mean()

mac
8c:85:90:16:0a:a4    6.912661
ac:9e:17:7d:31:e8    3.905111
d8:c4:6a:50:e3:b1    5.204745
f8:cf:c5:97:e0:9e    6.373128
Name: y, dtype: float64